## Competition Finals : `Access ? Granted : Denied`

The following notebook contains a general solution to the recently concluded PSHS-Samsung InfoTech Challenge.

### Problem statement
---

Siblings John and Jane communicate with each other through small encrypted computer networks. Usually, such networks are small and thus, simple to operate on. However, John has to go overseas to pursue his dreams on being the first man to solve the Collatz conjecture. Hence, the encrypted computer networks have multiplied in size and complexity.

Being far from home, John has developed measures to ensure that their system is encrypted. First, is that all of the computers in the network are closed. Second is that, John will only open half the total number of computers to prevent possible data tapping. However, due to percolation restrictions, the terminal computers may not be able to communicate all the time. When certain paths exist, John must be able to find the most cost-efficient path to communicate with his sister.

From what is given, it can be inferred that these siblings may be hiding information from network tapping systems. Tap Co. has developed a cheap solution to simplifying unknown computer networks into simple grids and using three signal receptors to determine the relative positions of network subsets by triangulation.

From the solution of Tap Co. it was found out that John uses a $15 x 15$ computer network, a 255 computer network, aside from the terminal computers used by the siblings, arranged in a $15 x 15$ graph. In this model, we will incorporate Julia's peculiar property of using column-major matrices. The "top-left" computer will be referred to as the node $(1, 1)$. Signal receptors are placed on nodes $(-1, 8), (16, -1)$, and $(16, 16)$.

Given sets of times for the signal to reach the three receptors, determine if John was able to communicate with Jane.

### Geometric Solution
---

The solution implemented below follows from the idea that the position of a point, relative to the positions of three points, can be determined using geometric methods. Three circles, centered at the reference points, can be drawn with their radii to be the appropriate distances to the point of interest where they all intersect. Presented in this [link][seis] is an application of _Triangulation_ in seismology.

For a computational representation of the solution, the appropriate standard equations of the circles must be constructed first from the given centers of the circles. Two equations can be constructed from choosing two pairs of circles and hence the coordinates of the point of interest may be obtained.

In the solution, Julia's implementation of a [matrix backsolve][bcks], is used to solve for the linear system. To learn more about matrix multiplication, [Khan Academy][khanm], provides a good explanation on matrix multiplication. The matrix backsolve is a "matrix" division. Let's say given $Ax = B$ then $x = A$ \\ $B$ where \\ is the backsolve operator.

A `Point` struct was defined to represent a point. A `TriangleSys` struct was defined to simplify triangulation input for fixed triangles. 

[seis]: http://www.qrg.northwestern.edu/projects/vss/docs/navigation/1-what-is-triangulation.html
[bcks]: https://stackoverflow.com/questions/45306412/how-to-use-the-backslash-operator-in-julia
[khanm]: https://www.khanacademy.org/math/precalculus/precalc-matrices/multiplying-matrices-by-matrices/v/multiplying-a-matrix-by-a-matrix

In [1]:
# the Point struct to represent a point
struct Point{T}
  x::T ## the left coordinate (vertical displacement from (0,0))
  y::T ## the right coordinate (horizontal displacement ftom (0, 0))
end

# the point should only have two coordinates
Point(x::Vector{Int}) = size(x) != 2 ? error("Vector should have two values") : Point(x...)

# the TriangleSys struct to represent the system of three signal receptors
struct TriangleSys{T}
  A::Point{T}
  B::Point{T}
  C::Point{T}
end

# the given TriangleSys object
const TAPCO = TriangleSys(Point(-1.0, 8.0), Point(16.0, -1.0), Point(16.0, 16.0))

# this uses the simplest solution to triangulation.
# this returns a Vector{T} of length 2 to represent the coordinates of the opened computer
function solvetriangle(B::TriangleSys{T}, r1::T, r2::T, r3::T) where T <: AbstractFloat
  rhs = [r1^2-r2^2+B.B.x^2-B.A.x^2+B.B.y^2-B.A.y^2; r2^2-r3^2+B.C.x^2-B.B.x^2+B.C.y^2-B.B.y^2]
  lhs = [B.B.x-B.A.x B.B.y-B.A.y; B.C.x-B.B.x B.C.y-B.B.y].*2
  return lhs\rhs
end

# function to calculate the distances of a Point with respect to the three points in a TriangleSys struct
function times(B::TriangleSys{T}, l::Point{T}) where T <: AbstractFloat
    return [hypot(B.A.x-l.x, B.A.y-l.y), hypot(B.B.x-l.x, B.B.y-l.y), hypot(B.C.x-l.x, B.C.y-l.y)]
end

times (generic function with 1 method)

### Percolation Solution
---

The following implementation is derived from the java implentation of the Percolation [solution][perc]. The problem and the `Quick Union` solution is given in the following book by Sedgewick and Wayne (Princeton) on [Algorithms][book] (p. 216-224).

[perc]: https://github.com/kristinpeterson/coursera_algorithms/tree/master/Percolation
[book]: http://www.albertstam.com/Algorithms.pdf

In [2]:
# the representation of a computer network (a Cell struct)
mutable struct Cell
    r::Int
    o::Bool
end

Cell(r::Int) = Cell(r, false)
function open(c::Cell)
    c.o = true
end
root(c::Cell) = c.r; isOpen(c::Cell) = c.o

# the representation of a network grid
mutable struct Percolation
    h::Int
    w::Int
    a::Vector{Cell}
end

# object constructor
Percolation(h, w) = Percolation(h, w, Cell.(Vector(1:h*w)))
# coordinate converter
c_vec(t::Percolation, h::Int, w::Int) = 0 < h < t.h+1 && 0 < w < t.w+1 ? t.h*(w-1)+h : throw(BoundsError(t))
# open cell
openT(t::Percolation, h::Int, w::Int) = open(t.a[c_vec(t, h, w)])

# check if cell is open in Percolation
isOpen(t::Percolation, h::Int, w::Int) = isOpen(t.a[c_vec(t, h, w)])

# get the "root" of the computer as discussed by the reference
function root(t::Percolation, h::Int, w::Int)
    ret = root(t.a[c_vec(t, h, w)])
    while ret != root(t.a[ret])
        ret = root(t.a[ret])
    end
    return ret
end

# connect the computers given their coordinates
function unionT(t::Percolation, h1::Int, w1::Int, h2::Int, w2::Int)
    if !isOpen(t, h1, w1) return
    else
        l = root(t.a[c_vec(t, h1, w1)]); r = root(t, h2, w2)
        t.a[c_vec(t, h1, w1)].r = l > r ? r : l
        t.a[c_vec(t, h2, w2)].r = t.a[c_vec(t, h1, w1)].r
    end
end

# connect open computers around a cell
function openunion(t::Percolation, h::Int, w::Int)
    openT(t, h, w)
    try unionT(t, h, w-1, h, w) end
    try unionT(t, h-1, w, h, w) end
    try unionT(t, h+1, w, h, w) end
    try unionT(t, h, w+1, h, w) end
end

# test if the John and Jane are connected
percolates(t::Percolation) = !isempty([root(t, i, t.w) for i=1:t.h if root(t, i, t.w) < t.h+1])

# print the computer system (roots of computers)
function printT(t::Percolation)
    for i=1:t.w
        for j=1:t.h
            print(root(t, i, j), ' ')
        end
        print('\n')
    end
end
         
# print the computer system (open computers)
function printO(t::Percolation)
    for i=1:t.w
        for j=1:t.h
            print(isOpen(t, i, j) ? 1 : 0, ' ')
        end
        print('\n')
    end
end

printO (generic function with 1 method)

Print a random percolating system and print the system (`printO`)

In [3]:
t = Percolation(15, 15)
a = Array{Float64}(2)
while !percolates(t)
    c = Int.(ceil.(rand(2).*15))
    if !isOpen(t, c...)
        openunion(t, c...); a = hcat(a, c)
    else
        continue
    end
end
printO(t); a; print(size(a)[2]/225)

1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 
1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 
1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 
1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 
1 1 0 1 1 1 1 0 0 0 0 1 0 1 1 
0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 0 0 0 1 0 1 1 
0 1 1 0 0 0 0 0 1 1 0 1 0 1 1 
0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 
0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 
1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 
1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 
1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 
0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 
1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 
0.6977777777777778

### Solutions to the problems given at the event
---

Each computer has a different dataset given to them representing the distances traveled by the signals to reach the receptors. All solutions are given below.

Import the needed package for loading CSVs (`CSV.jl`)

In [4]:
using CSV

In [5]:
w = Array{Float64}(CSV.read("cp_01/in.csv"))
t = Percolation(15, 15)
for i=1:size(w)[1]
  openunion(t, Int.(round.(solvetriangle(TAPCO, w[i,:]...)))...)
end
print(percolates(t))

true

The system percolates!

In [6]:
printO(t); println(); printT(t)

1 1 0 1 1 1 0 1 0 0 0 1 1 1 0 
0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 
0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 
0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 
1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 
0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 
0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 
1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 
1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 
1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 
1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 
0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 
1 0 1 0 0 0 1 0 0 1 1 1 0 1 1 

1 1 31 46 46 46 91 106 121 136 151 166 166 166 211 
2 1 32 47 46 77 92 106 106 106 106 167 166 166 166 
3 1 1 48 46 78 93 106 123 106 106 106 183 166 166 
4 1 1 49 1 79 94 106 106 106 106 106 184 166 214 
5 20 1 1 1 1 1 110 106 106 106 106 185 166 166 
5 5 5 1 5 5 1 1 126 1 1 1 186 166 166 
7 5 5 1 5 1 1 1 1 1 1 1 187 166 217 
8 5 38 5 5 5 98 1 128 1 158 1 188 166 1 
9 24 5 5 5 5 99 1 1 1 159 1 1 1 1 
10 10 40 55 5 5 5 5 130 1 160 117 190 1 1 
10 10 41 10 5 5 5 116 117 146 117 117 117 117 1 
10 10 10 10 5 87 102 117 117 117 117 

The cell above shows the expected output of the group given the dataset. There are numerous paths from the left to the right parts of the network. The participants were only required to show one. Open computers are shown as `1` in the above output. _Roots_ of computers are displayed in the below grid.

In [7]:
w = Array{Float64}(CSV.read("cp_08/in.csv"))
t = Percolation(15, 15)
for i=1:size(w)[1]
  openunion(t, Int.(round.(solvetriangle(TAPCO, w[i,:]...)))...)
end
print(percolates(t))

true

The system percolates!

In [8]:
printO(t); println(); printT(t)

0 0 1 0 0 0 0 1 1 1 1 1 1 0 1 
1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 
0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 
1 1 0 1 0 0 0 0 1 1 1 0 1 0 0 
1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 
0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 
1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 
1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 
1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 
0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 
1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 
0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 0 0 1 1 0 
1 0 1 0 1 1 0 0 0 1 1 0 1 1 0 
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 

1 16 31 46 61 76 91 106 106 106 106 106 106 196 211 
2 2 32 4 62 77 92 106 122 106 152 106 106 197 106 
3 4 4 4 4 4 93 108 110 110 110 168 106 106 106 
4 4 34 4 64 79 94 109 110 110 110 169 106 199 214 
4 4 35 4 65 4 95 110 110 110 110 110 110 200 215 
6 4 4 4 4 4 96 97 110 110 110 171 110 201 215 
7 4 37 52 67 82 97 97 97 142 110 110 110 202 217 
7 23 7 53 68 83 98 113 110 143 158 173 110 110 110 
7 7 7 54 7 7 99 114 110 110 110 110 110 110 219 
10 7 7 7 7 7 100 114 110 145 160 110 110 110 220 
11 11 41 7 7 7 7 7 114 146 7 7 19

The cell above shows the expected output of the group given the dataset. There are numerous paths from the left to the right parts of the network. The participants were only required to show one. Open computers are shown as `1` in the above output. _Roots_ of computers are displayed in the below grid.

In [9]:
w = Array{Float64}(CSV.read("cp_14/in.csv"))
t = Percolation(15, 15)
for i=1:size(w)[1]
  openunion(t, Int.(round.(solvetriangle(TAPCO, w[i,:]...)))...)
end
print(percolates(t))

false

The system does not percolate! See [Denied][deny] below for the solution.

[deny]: #Access-Denied

In [10]:
w = Array{Float64}(CSV.read("cp_18/in.csv"))
t = Percolation(15, 15)
for i=1:size(w)[1]
  openunion(t, Int.(round.(solvetriangle(TAPCO, w[i,:]...)))...)
end
print(percolates(t))

true

The system percolates!

In [11]:
printO(t); println(); printT(t)

1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 
1 1 1 1 1 0 1 1 0 0 0 1 0 0 1 
1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 
1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 
1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 
1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 
1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 
0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 
0 1 0 1 1 1 1 0 0 1 0 1 0 1 0 
0 0 1 1 0 0 0 1 1 1 0 1 1 0 1 
1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 
1 0 1 0 0 0 1 1 1 1 0 1 0 1 0 
0 0 1 0 1 1 0 1 1 1 0 1 0 1 1 
1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 
0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 

1 1 1 1 1 1 1 106 121 136 151 151 181 196 183 
1 1 1 1 1 77 1 1 122 137 152 151 182 197 183 
1 18 33 48 1 1 1 1 123 138 153 168 183 183 183 
1 1 1 1 1 1 1 1 124 139 154 154 184 199 183 
1 1 1 1 65 1 1 1 1 140 154 170 1 1 215 
1 1 36 1 1 1 96 1 1 1 1 1 1 1 216 
1 1 1 52 67 1 1 1 1 1 157 1 1 1 1 
8 23 1 1 1 83 1 1 1 143 158 158 1 203 218 
9 24 39 40 1 40 1 114 129 115 159 115 189 204 219 
10 25 40 40 70 85 100 115 115 115 160 115 115 205 220 
11 11 41 40 71 86 101 102 115 115 115 115 115 115 221 
11 27 42 57 72 87 102 102 102 115 

The cell above shows the expected output of the group given the dataset. There are numerous paths from the left to the right parts of the network. The participants were only required to show one. Open computers are shown as `1` in the above output. _Roots_ of computers are displayed in the below grid.

In [12]:
w = Array{Float64}(CSV.read("cp_28/in.csv"))
t = Percolation(15, 15)
for i=1:size(w)[1]
  openunion(t, Int.(round.(solvetriangle(TAPCO, w[i,:]...)))...)
end
print(percolates(t))

false

The system does not percolate! See [Denied][deny] below for the solution.

[deny]: #Access-Denied

### Access-Denied
---

We will randomly generate $500$ percolating systems and we will determine the mean fraction and standard error of opened computers in the system for John and Jane to communicate.

In [13]:
q = Vector{Float64}(); l = 0
while l < 500
    t = Percolation(15, 15); n = 0
    while !percolates(t)
        c = Int.(ceil.(rand(2).*15))
        if !isOpen(t, c...)
            openunion(t, c...); n+=1;
        else
            continue
        end
    end
    push!(q, n); n=0; l+=1
end
print(mean(q)/225, ' ', std(q)/225)

0.6982133333333334 0.08125544464645335

This means that about $70\% \pm 8\%$ of computers, on the average, must be open for the system to be percolating.